In [ ]:
# https://towardsdatascience.com/examples-of-using-apache-spark-with-pyspark-using-python-f36410457012

In [1]:
import sys

In [2]:
sys.path

['D:\\Projects\\TG_GitHub\\python_pyspark',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\python37.zip',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\DLLs',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\lib',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37',
 '',
 'C:\\Users\\DELL\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\lib\\site-packages',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\win32',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\win32\\lib',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\Pythonwin',
 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\DELL\\.ipython']

In [3]:
import findspark

In [4]:
findspark.init('C:\spark\spark-2.4.5-bin-hadoop2.7/')

In [5]:
from pyspark import SparkContext
from pyspark import SparkConf,SparkContext
import numpy as np

In [6]:
sc=SparkContext(master="local[3]")

In [10]:
conf=SparkConf().setMaster("local")
sc=SparkContext(conf = conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[3]) created by __init__ at <ipython-input-9-207199eefb6e>:1 

In [7]:
lst = np.random.randint(0,10,20)
A = sc.parallelize(lst)

In [8]:
type(A)

pyspark.rdd.RDD

In [9]:
A.collect()

[9, 0, 2, 6, 2, 5, 9, 4, 2, 3, 8, 5, 8, 4, 3, 5, 5, 6, 5, 8]

In [10]:
A.glom().collect()

[[9, 0, 2, 6, 2, 5], [9, 4, 2, 3, 8, 5], [8, 4, 3, 5, 5, 6, 5, 8]]

In [11]:
sc.stop()
sc=SparkContext(master="local[2]")
A = sc.parallelize(lst)
A.glom().collect()

[[9, 0, 2, 6, 2, 5, 9, 4, 2, 3], [8, 5, 8, 4, 3, 5, 5, 6, 5, 8]]

In [12]:
A.first()

9

In [13]:
A.take(3)

[9, 0, 2]

In [16]:
A_distinct=A.distinct()
A_distinct.collect()

[0, 2, 6, 4, 8, 9, 5, 3]

In [17]:
# To sum all the elements use reduce method
A.reduce(lambda x,y:x+y)

99

In [18]:
A.sum()

99

In [19]:
# Finding maximum element by reduce
A.reduce(lambda x,y: x if x > y else y)

9

In [20]:
# Finding longest word in a blob of text
words = 'These are some of the best Macintosh computers ever'.split(' ')
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)

'computers'

In [24]:
# Use filter for logic-based filtering
# Return RDD with elements (greater than zero) divisible by 3 
A.filter(lambda x:x%3==0 and x!=0).collect()

[9, 6, 9, 3, 3, 6]

In [25]:
# Write regular Python functions to use with reduce()
def largerThan(x,y):
    """ Returns the last word among the longest words in a list """ 
    if len(x)> len(y):
        return x 
    elif len(y) > len(x): 
        return y 
    else: 
        if x < y: 
            return x 
        else: 
            return y 
wordRDD.reduce(largerThan) 

'Macintosh'

In [26]:
# Mapping operation with a lambda function with PySpark
B = A.map(lambda x:x*x) 
B.collect()

[81, 0, 4, 36, 4, 25, 81, 16, 4, 9, 64, 25, 64, 16, 9, 25, 25, 36, 25, 64]

In [27]:
# Mapping with a regular Python function in PySpark
def square_if_odd(x): 
    """ Squares if odd, otherwise keeps the argument unchanged """ 
    if x%2==1: 
        return x*x 
    else: 
        return x 
A.map(square_if_odd).collect()

[81, 0, 2, 6, 2, 25, 81, 4, 2, 9, 8, 25, 8, 4, 9, 25, 25, 6, 25, 8]

In [28]:
# groupby returns a RDD of grouped elements (iterable) as per a given group operation
result = A.groupBy(lambda x:x%2).collect() 
sorted([(x, sorted(y)) for (x, y) in result])

[(0, [0, 2, 2, 2, 4, 4, 6, 6, 8, 8, 8]), (1, [3, 3, 5, 5, 5, 5, 5, 9, 9])]

In [29]:
result

[(0, <pyspark.resultiterable.ResultIterable at 0x20b2b4654c8>),
 (1, <pyspark.resultiterable.ResultIterable at 0x20b2b465448>)]

In [32]:
sorted(result[0][1])

[0, 2, 2, 2, 4, 4, 6, 6, 8, 8, 8]

In [33]:
# Couldn't understand
B.histogram([x for x in range(0,100,10)])

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [6, 2, 5, 2, 0, 0, 3, 0, 2])

In [34]:
B.collect()

[81, 0, 4, 36, 4, 25, 81, 16, 4, 9, 64, 25, 64, 16, 9, 25, 25, 36, 25, 64]

### Set operations

In [6]:
#sc.stop()
sc = SparkContext(master="local[2]")

In [7]:
# Make a RDD with 1 million elements
%time rdd1 = sc.parallelize(range(100000))

Wall time: 705 ms


In [8]:
# Some computing function — taketime
from math import cos 
def taketime(x): 
    [cos(j) for j in range(1000)] 
    return cos(x)

In [9]:
# Check how much time is taken by taketime function
%time taketime(2)

Wall time: 0 ns


-0.4161468365471424

In [10]:
# Now do the map operation on the function
%time interim = rdd1.map(lambda x: taketime(x))

Wall time: 0 ns


In [11]:
# The actual execution by reduce method
%time print('output =',interim.reduce(lambda x,y:x+y))

output = 1.0323992715959767
Wall time: 29.5 s


In [12]:
%time print(interim.filter(lambda x:x>0).count())

50001
Wall time: 28.7 s


In [13]:
# Caching to reduce computation time on similar operation (spending memory)
%time interim = rdd1.map(lambda x: taketime(x)).cache()

Wall time: 14.9 ms


In [14]:
%time print('output =',interim.reduce(lambda x,y:x+y))

output = 1.0323992715959767
Wall time: 30.1 s


In [15]:
# Now run the same filter method with the help of cached result
%time print(interim.filter(lambda x:x>0).count())

50001
Wall time: 1.2 s
